In [ ]:
import face_recognition as fr
import os
import pickle
import sys
import cv2, dlib, math
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt

In [ ]:
base =  r".\dataset"
names = os.listdir(base)                # names of people
person_face_encodings = dict()


for name in names:
    person_face_encodings[name] = []
    for file_name in os.listdir(base+'/'+name):
        img = fr.load_image_file(base+'/'+name+'/'+file_name)
        encodings = fr.face_encodings(img, model='large')
        if len(encodings):
            person_face_encodings[name].append(encodings[0])

try:
    f = open("./important files/face_encodings.pickle", 'wb')
    pickle.dump(person_face_encodings, f)
    f.close()
except:
    print("Something went wrong!")
    os._exit(1)

In [ ]:
try:
    f = open("./important files/face_encodings.pickle", 'rb')
    person_face_encodings = pickle.load(f)
    f.close()
except:
    print("Something went wrong!")
    os._exit(1)

In [ ]:
target = dict()

a = 0
for p in list(person_face_encodings.keys()):
    target[p] = a
    a += 1
reverse_target = dict([(v,k) for k,v in target.items()])
target

In [ ]:
X = []
y = []

for k,v in person_face_encodings.items():
    for i in v:
        X.append(i)
        y.append(target[k])

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)

knn.fit(X,y)

In [ ]:
knn.score(X,y)

In [ ]:
def face_detection_2(img):
    detector = dlib.get_frontal_face_detector()
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray_img)
    cropped_faces = []
    take = 7  # increase the size of the bounding box
    for face in faces:
        cropped_faces.append(img[ face.top()-take :face.bottom()+take, face.left()-take:face.right()+take , :].copy())
    for face in faces:
        cv2.rectangle(img,(face.left()-5, face.top()-5), (face.right()+5, face.bottom()+5) ,(0,255,0), 5)
    return cropped_faces , img

def recognize_faces(img_path):
    try:
        file = open("./face_encodings.pickle", 'rb')
        person_face_encodings = pickle.load(file)
        file.close()
    except:
        print("Something went wrong!")
        os._exit(1)
    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    img = np.uint8(img)
    cropped_faces, img = face_detection_2(img)
    # print(len(cropped_faces), cropped_faces[0].shape)
    # plt.imshow(img)
    # plt.show()

    ans = dict()
    face_vs_name = []
    for i,face in enumerate(cropped_faces):
        # plt.imshow(face)
        # plt.show()
        unknown_encoding = fr.face_encodings(face ,num_jitters=3 , model='large')
        if len(unknown_encoding):
            ans = knn.predict(unknown_encoding)
            face_vs_name.append(reverse_target[ans[0]])
        else :
            face_vs_name.append("~ENCODING")

    fig, axes = plt.subplots( math.ceil(len(face_vs_name)/2) , 2 )
    for i,face in enumerate(cropped_faces):
        axes.ravel()[i].imshow(face)
        axes.ravel()[i].set_title(face_vs_name[i])
        axes.ravel()[i].axis('off')
    fig.tight_layout()
    plt.show()

    return face_vs_name

In [ ]:
recognize_faces(r"./test images/1.jpg")

In [ ]:
recognize_faces(r"./test images/6.jpg")

In [ ]:
try:
    f = open("./important files/facenet_knn.pickle", 'wb')
    pickle.dump(knn, f)
    f.close()
except:
    print("Something went wrong!")
    os._exit(1)

In [ ]:
try:
    f = open("./important files/facenet_knn.pickle", 'rb')
    model = pickle.load(f)
    f.close()
except:
    print("Something went wrong!")
    os._exit(1)